In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from google.oauth2 import service_account
from google.colab import drive
import datetime as dt

In [ ]:
# Read from google drive and create the dataframes
path1='/content/drive/MyDrive/Proyecto_Final_BC/Raw_Data/Collection1.xlsx'
path2='/content/drive/MyDrive/Proyecto_Final_BC/Raw_Data/Collection2.xlsx'
path3='/content/drive/MyDrive/Proyecto_Final_BC/Raw_Data/Claim1.xlsx'
path4='/content/drive/MyDrive/Proyecto_Final_BC/Raw_Data/Claim2.xlsx'
path5='/content/drive/MyDrive/Proyecto_Final_BC/Raw_Data/Chargeback1.xlsx'
path6='/content/drive/MyDrive/Proyecto_Final_BC/Raw_Data/Chargeback2.xlsx'
path7='/content/drive/MyDrive/Proyecto_Final_BC/Raw_Data/Magento1.xlsx'
path8='/content/drive/MyDrive/Proyecto_Final_BC/Raw_Data/Magento2.csv'
colle1_df=pd.read_excel(path1)
colle2_df=pd.read_excel(path2)
claim1_df=pd.read_excel(path3)
claim2_df=pd.read_excel(path4)
chrgbk1_df=pd.read_excel(path5)
chrgbk2_df=pd.read_excel(path6)
magent1_df=pd.read_excel(path7)
magent2_df=pd.read_csv(path8)

In [ ]:
# Joining all dataframes
magento_df=pd.concat([magent1_df,magent2_df])
collection_df=pd.concat([colle1_df,colle2_df])
claims_df=pd.concat([claim1_df,claim2_df])
chrgback_df=pd.concat([chrgbk1_df,chrgbk2_df])

In [ ]:
#Keeping relevant columns and renaming columns
magento_df2=magento_df[['"ID"','Billing Address','Shipping Address','Shipping Information','Shipping and Handling','Order Items']]
magento_df2=magento_df2.rename(columns={'"ID"':'external_reference','Billing Address':'Bill_Address','Shipping Address':'Shp_Address','Shipping Information':'Shp_Carrier','Shipping and Handling':'Shipping','Order Items':'Items'})
collection_df2=collection_df[['Fecha de compra (date_created)','E-mail de la contraparte (counterpart_email)','Código de referencia (external_reference)','Número de operación de Mercado Pago (operation_id)','Estado de la operación (status)','Detalle del estado de la operación (status_detail)','Valor del producto (transaction_amount)','Cuotas (installments)','Medio de pago (payment_type)']]
collection_df2=collection_df2.rename(columns={'Fecha de compra (date_created)':'date_created','Fecha de acreditación (date_approved)':'date_approved','E-mail de la contraparte (counterpart_email)':'e-mail','Código de referencia (external_reference)':'external_reference','Número de operación de Mercado Pago (operation_id)':'operation_id','Estado de la operación (status)':'status','Detalle del estado de la operación (status_detail)':'status_detail','Valor del producto (transaction_amount)':'transaction_amount','Cuotas (installments)':'installments','Medio de pago (payment_type)':'payment_type'})
claims_df2=claims_df[['Fecha de creación del reclamo (date_created)','Detalle del estado del reclamo (status_detail)','Número de operación de Mercado Pago (operation_id)']]
claims_df2=claims_df2.rename(columns={'Fecha de creación del reclamo (date_created)':'claim_date_created','Detalle del estado del reclamo (status_detail)':'claim_status_detail','Número de operación de Mercado Pago (operation_id)':'operation_id'})
chrgback_df2=chrgback_df[['Fecha de Creación del contracargo (date_created)','Detalle del estado del contracargo (status_detail)','Número de operación de Mercado Pago (operation_id)']]
chrgback_df2=chrgback_df2.rename(columns={'Fecha de Creación del contracargo (date_created)':'claim_date_created','Detalle del estado del contracargo (status_detail)':'claim_status_detail','Número de operación de Mercado Pago (operation_id)':'operation_id'})



In [ ]:
# Getting Fraud Dataframe and filling up NaN values with 'Claim' Value
fraud_df=pd.concat([chrgback_df2,claims_df2])
fraud_df['claim_status_detail'].fillna('claim',inplace=True)

In [ ]:
# Convert object columns to datetime columns
collection_df2['date_created']=pd.to_datetime(collection_df2['date_created'])
fraud_df['claim_date_created']=pd.to_datetime(fraud_df['claim_date_created'])

In [ ]:
# Getting the hour number, month and dayofweek (0=mon - 6=sun)
collection_df2['hour_created']=collection_df2['date_created'].dt.hour
collection_df2['DOW_created']=collection_df2['date_created'].dt.dayofweek
collection_df2['month_created']=collection_df2['date_created'].dt.month

In [ ]:
# Getting the external_reference number and casting to integer
collection_df2['external_reference']=collection_df2.external_reference.str.extract('(\d+)').astype(str).astype(int)

In [ ]:
collection_df2.head()

,date_created,e-mail,external_reference,operation_id,status,status_detail,transaction_amount,installments,payment_type,hour_created,DOW_created,month_created
0,2022-05-17 17:02:12,34,375285,22439390268,approved,accredited,4549.0,1,debit_card,17,1,5
1,2022-05-17 17:00:42,35,375284,22439310665,approved,accredited,4549.0,1,debit_card,17,1,5
2,2022-05-17 13:23:07,36,375213,22433076416,rejected,cc_rejected_high_risk,6378.0,3,credit_card,13,1,5
3,2022-05-17 13:26:43,37,375217,22433120457,rejected,cc_rejected_high_risk,6378.0,1,credit_card,13,1,5
4,2022-05-17 15:01:25,38,375235,22435577370,approved,accredited,6689.0,9,credit_card,15,1,5


In [ ]:
# Getting the hour number, month and dayofweek (0=mon - 6=sun) from fraud dataframe
fraud_df['claim_hour_created']=fraud_df['claim_date_created'].dt.hour
fraud_df['claim_DOW_created']=fraud_df['claim_date_created'].dt.dayofweek
fraud_df['claim_month_created']=fraud_df['claim_date_created'].dt.month

In [ ]:
fraud_df.head()

,claim_date_created,claim_status_detail,operation_id,claim_hour_created,claim_DOW_created,claim_month_created
0,2022-05-16 10:45:25,ppv_valid,20232353481,10,0,5
1,2022-10-05 09:55:26,ppv_valid,21424519055,9,2,10
2,2022-04-20 06:05:28,ppv_valid,21469947214,6,2,4
3,2022-04-19 06:05:10,ppv_valid,21415488185,6,1,4
4,2022-03-18 06:05:15,lost,20836933063,6,4,3


In [ ]:
# Getting the bill_address and shp_address zipcode
magento_df2['bill_zipcode']=magento_df2.Bill_Address.str.extract('(\d{5})')
magento_df2['shp_zipcode']=magento_df2.Shp_Address.str.extract('(\d{5})')
# magento_df2.dropna(['bill_zipcode'], inplace=True)

In [ ]:
magento_df2.head()

,external_reference,Bill_Address,Shp_Address,Shp_Carrier,Shipping,Items,bill_zipcode,shp_zipcode
0,68857,"Casa Color azul turquesa puerta negra, primer ...","Casa Color azul turquesa puerta negra, primer ...",Estafeta - Servicio Express (Entrega promedio ...,0,<div>2 x [TELM131] Xiaomi Earbuds Negros </div>,36030,36030
1,68858,Entre Av pantitlan y Calle Glorieta fuente de ...,Entre Av pantitlan y Calle Glorieta fuente de ...,Estafeta - Servicio Express (Entrega promedio ...,0,<div>1 x [TELM0912831] Hisense E40 32GB Azul <...,57700,57700
2,68859,"Entre calle Alemania y calle Francia ,Brasil\n...","Entre calle Alemania y calle Francia ,Brasil\n...",Estafeta - Servicio Express (Entrega promedio ...,0,<div>1 x [TELM611] Xiaomi Mi Band 5 Negro </div>,47020,47020
3,68860,"casa blanca con rejas cafes, a un costado de u...","casa blanca con rejas cafes, a un costado de u...",Estafeta - Servicio Express (Entrega promedio ...,0,<div>1 x [TEL000111] Xiaomi Poco X3 128GB Gris...,85930,85930
4,68861,Casa verde de 2 pisos con cortina y porton bla...,Casa verde de 2 pisos con cortina y porton bla...,Estafeta - Servicio Express (Entrega promedio ...,0,<div>1 x [TELM091112] Xiaomi Poco X3 NFC 128GB...,55120,55120


In [ ]:
# Merging all data frames
collection_df3=pd.merge(collection_df2,fraud_df,how='left',on='operation_id')
whole_collection_df=pd.merge(collection_df3,magento_df2, how='left', on='external_reference')

In [ ]:
# Filling NaN values with 'None" in the Status Detail for Whole collection
whole_collection_df['claim_status_detail'].fillna('none',inplace=True)

In [ ]:
# Setting a fraud flag for 0 = non-fraud and 1 = fraud
whole_collection_df.loc[whole_collection_df['claim_status_detail'] == 'none', 'fraud_flag'] = 0  
whole_collection_df.loc[whole_collection_df['claim_status_detail'] != 'none', 'fraud_flag'] = 1  

In [ ]:
#Dropping empty rows for bill zipcode
whole_collection_df.dropna(subset=['bill_zipcode'],inplace=True)
whole_collection_df.dropna(subset=['shp_zipcode'],inplace=True)


In [ ]:
whole_collection_df['bill_zipcode']=whole_collection_df.bill_zipcode.astype(str).astype(int)
whole_collection_df['shp_zipcode']=whole_collection_df.shp_zipcode.astype(str).astype(int)

In [ ]:

whole_collection_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19630 entries, 0 to 19717
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date_created         19630 non-null  datetime64[ns]
 1   e-mail               19630 non-null  int64         
 2   external_reference   19630 non-null  object        
 3   operation_id         19630 non-null  int64         
 4   status               19630 non-null  object        
 5   status_detail        19630 non-null  object        
 6   transaction_amount   19630 non-null  float64       
 7   installments         19630 non-null  int64         
 8   payment_type         19630 non-null  object        
 9   hour_created         19630 non-null  int64         
 10  DOW_created          19630 non-null  int64         
 11  month_created        19630 non-null  int64         
 12  claim_date_created   332 non-null    datetime64[ns]
 13  claim_status_detail  19630 non-

In [ ]:
#Saving dataframe into Drive
whole_collection_df.to_csv('/content/drive/MyDrive/Proyecto_Final_BC/Whole_Collection.csv')